In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir('../')

In [3]:
import numpy as np
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from torch.utils.data import TensorDataset, DataLoader
from torch_geometric.data import DataLoader, Dataset
from copy import deepcopy
import random
import pickle
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Subset, random_split
import glob
from PIL import Image
from collections import Counter
import torch_geometric
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
import torch.nn.functional as F
from torch.nn import Linear, ReLU, BatchNorm1d, Module, Sequential
from torch_geometric.nn import global_mean_pool, global_max_pool, global_sort_pool, global_add_pool
from torch_scatter import scatter
from torch_geometric.data import Data
from torch_geometric.utils import to_undirected
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
import wandb

In [4]:
from experiments import hierarchy
from experiments.cbm_models import *
from experiments.dataset import *

## Load Hierarchies in 

In [5]:
mnist_hierarchy = hierarchy.Hierarchy()
mnist_hierarchy.from_array(np.array(np.load(open("results/hierarchies/label_mnist.npy","rb"))),hierarchy.mnist_attributes)
mnist_groups = hierarchy.traverse_hierarchy(mnist_hierarchy.root_split)
mnist_groups_idx = [[hierarchy.mnist_attributes.index(j) for j in group] for group in mnist_groups]

In [6]:
cub_hierarchy = hierarchy.Hierarchy()
cub_hierarchy.from_array(np.array(np.load(open("results/hierarchies/label_cub.npy","rb"))),hierarchy.cub_attributes)
cub_groups = hierarchy.traverse_hierarchy(cub_hierarchy.root_split)
cub_groups_idx = [[hierarchy.cub_attributes.index(j) for j in group] for group in cub_groups]

## Load Data

In [7]:
cub_train_independent, cub_val_independent = load_cub_independent()
cub_train_sequential, cub_val_sequential = load_cub_sequential()

In [8]:
cub_train_cem, cub_val_cem = load_cub_cem()

In [9]:
cub_related_labels = get_related_scores(np.load(open("concept_vectors/labels_43.npy","rb")))
embedding_matrix = np.load(open("concept_vectors/labels_43.npy","rb"))
sim_matrix = cosine_similarity(embedding_matrix)

In [10]:
edge_attr, edge_index = get_attr_index(cub_groups,hierarchy.cub_attributes,sim_matrix,use_random=False)

In [11]:
cub_train_independent_graph = tensor_to_graph(cub_train_independent,edge_attr,edge_index)
cub_val_independent_graph = tensor_to_graph(cub_val_independent,edge_attr,edge_index)
cub_train_sequential_graph = tensor_to_graph(cub_train_sequential,edge_attr,edge_index)
cub_val_sequential_graph = tensor_to_graph(cub_val_sequential,edge_attr,edge_index)
cub_train_cem_graph = tensor_to_graph(cub_train_cem,edge_attr,edge_index)
cub_val_cem_graph = tensor_to_graph(cub_val_cem,edge_attr,edge_index)

/rds/user/njr61/hpc-work/anaconda3/envs/gnn/lib/python3.9/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [12]:
x,y,c = next(iter(cub_train_independent))
bottleneck_size = c.shape[0]
output_classes = 10 if bottleneck_size == 21 else 200

## Evaluate Models

In [37]:
scores_by_model_type = {}
bottleneck_size = 112
output_classes = 200

In [107]:
for model_type in ['gnn','gnn_basic','mlp_bool','mlp_group','mlp','gnn_bool']:
    lr_values = [1e-4,5e-4,1e-3,5e-3,1e-2,5e-2,1e-1]
    
    train = cub_train_sequential
    val = cub_val_sequential
    
    if 'gnn' in model_type:
        edge_attr, edge_index, train, val = get_dataset_graph_cub(model_type,cub_groups,hierarchy.cub_attributes,sim_matrix
                                                                 , train, val)
        
    hyperparameters = {'lr': .01, 'epochs': 5, 'num_layers': 1, 'emb_dim': 4, 
                       'in_dim': 112, 'out_dim': 200, 
                       'edge_dim': 1,'sim_matrix': sim_matrix, 'attributes': hierarchy.cub_attributes, 'group': cub_groups, 
                       'indexes': cub_groups_idx}
    
    if 'gnn' in model_type:
        hyperparameters['emb_size'] = 64
    
    hyperparameters = update_hyperparameters_graph(model_type,hyperparameters)

    best_lr = find_optimal_lr(model_type,lr_values,hyperparameters,train,val)
    
    hyperparameters['lr'] = best_lr
    hyperparameters['epochs'] = 20
        
    score_list = []
    print("Finished!")    
    for trial in range(1):
        train = cub_train_sequential
        val = cub_val_sequential

        if 'gnn' in model_type:
            edge_attr, edge_index, train, val = get_dataset_graph_cub(model_type,
                                                                      cub_groups,hierarchy.cub_attributes,sim_matrix, train, val)
                        
        hyperparameters = update_hyperparameters_graph(model_type,hyperparameters)
        
        model  = initialize_model(model_type,hyperparameters,use_wandb=True)
        model = train_model(model,model_type,train,val,hyperparameters,use_wandb=True)
        score = eval_model(model,model_type,val)
        score_list.append(score)
    
    scores_by_model_type[model_type] = np.mean(score_list)
    
    print("Score for {}, {} +- {}".format(model_type,np.mean(score_list),np.std(score_list)))

Finished!
Score for gnn, 0.6606320142745972 +- 0.14176370203495026
Finished!
Score for gnn_basic, 0.739497184753418 +- 0.04796473681926727


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type    | Params
------------------------------------
0 | relu    | ReLU    | 0     
1 | softmax | Softmax | 0     
2 | sigmoid | Sigmoid | 0     
3 | fc      | Linear  | 6.4 K 
------------------------------------
6.4 K     Trainable params
0         Non-trainable params
6.4 K     Total params
0.026     Total estimated model params size (MB)
`Trainer.fit` stopped: `max_epochs=5` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type    | Params
------------------------------------
0 | relu    | ReLU    | 0     
1 | softmax | Softmax | 0     
2 | sigmoid | Sigmoid | 0     
3 | fc      | Linear  | 6.4 K 
------------------------------------
6.4 K     Trainable params
0         Non-trainable params
6.4 K     T

Finished!


`Trainer.fit` stopped: `max_epochs=20` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type    | Params
------------------------------------
0 | relu    | ReLU    | 0     
1 | softmax | Softmax | 0     
2 | sigmoid | Sigmoid | 0     
3 | fc      | Linear  | 6.4 K 
------------------------------------
6.4 K     Trainable params
0         Non-trainable params
6.4 K     Total params
0.026     Total estimated model params size (MB)
`Trainer.fit` stopped: `max_epochs=20` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type    | Params
------------------------------------
0 | relu    | ReLU    | 0     
1 | softmax | Softmax | 0     
2 | sigmoid | Sigmoid | 0     
3 | fc      | Linear  | 6.4 K 
------------------------------------
6.4 K     Trainable 

Score for mlp_bool, 0.009599332325160503 +- 0.005514870362340228


...

Score for mlp_group, 0.029883137531578542 +- 0.0802880019780189


`Trainer.fit` stopped: `max_epochs=5` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type    | Params
------------------------------------
0 | relu    | ReLU    | 0     
1 | softmax | Softmax | 0     
2 | sigmoid | Sigmoid | 0     
3 | fc      | Linear  | 22.6 K
------------------------------------
22.6 K    Trainable params
0         Non-trainable params
22.6 K    Total params
0.090     Total estimated model params size (MB)
`Trainer.fit` stopped: `max_epochs=5` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type    | Params
------------------------------------
0 | relu    | ReLU    | 0     
1 | softmax | Softmax | 0     
2 | sigmoid | Sigmoid | 0     
3 | fc      | Linear  | 22.6 K
------------------------------------
22.6 K    Trainable pa

Finished!


`Trainer.fit` stopped: `max_epochs=20` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type    | Params
------------------------------------
0 | relu    | ReLU    | 0     
1 | softmax | Softmax | 0     
2 | sigmoid | Sigmoid | 0     
3 | fc      | Linear  | 22.6 K
------------------------------------
22.6 K    Trainable params
0         Non-trainable params
22.6 K    Total params
0.090     Total estimated model params size (MB)
`Trainer.fit` stopped: `max_epochs=20` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type    | Params
------------------------------------
0 | relu    | ReLU    | 0     
1 | softmax | Softmax | 0     
2 | sigmoid | Sigmoid | 0     
3 | fc      | Linear  | 22.6 K
------------------------------------
22.6 K    Trainable 

Score for mlp, 0.8971619248390198 +- 0.0008990219399868032
Finished!
Score for gnn_bool, 0.10975096374750137 +- 0.09455262124538422


In [108]:
scores_by_model_type

{'mlp': 0.8971619248390198,
 'mlp_group': 0.029883137531578542,
 'mlp_bool': 0.009599332325160503,
 'mlp_3sat': 0.6100000143051147,
 'gnn': 0.660632,
 'gnn_basic': 0.7394972,
 'gnn_bool': 0.109750964}

In [16]:
for model_type in ['mlp','gnn_bool_cem','gnn','gnn_basic','mlp_bool','mlp_group']:
    lr_values = [1e-4,5e-4,1e-3,5e-3,1e-2,5e-2,1e-1]
    
    train = cub_train_cem
    val = cub_val_cem

    if 'gnn' in model_type:
        edge_attr, edge_index, train, val = get_dataset_graph_cub(model_type,cub_groups,hierarchy.cub_attributes,sim_matrix
                                                                 , train, val)
        
    hyperparameters = {'lr': .01, 'epochs': 5, 'num_layers': 1, 'emb_dim': 16, 
                       'in_dim': 112, 'out_dim': 200, 
                       'edge_dim': 1,'sim_matrix': sim_matrix, 'attributes': hierarchy.cub_attributes, 'group': cub_groups, 
                       'indexes': cub_groups_idx}
    
    if 'gnn' in model_type:
        hyperparameters['emb_size'] = 112
    
    hyperparameters = update_hyperparameters_graph(model_type,hyperparameters)

    if 'mlp' in model_type:
        hyperparameters['bottleneck_size'] *= 16

    
    best_lr = find_optimal_lr(model_type,lr_values,hyperparameters,train,val)
    
    hyperparameters['lr'] = best_lr
    hyperparameters['epochs'] = 20
        
    score_list = []
    print("Finished!")
    
    for trial in range(10):
        train = cub_train_cem
        val = cub_val_cem

        if 'gnn' in model_type:
            edge_attr, edge_index, train, val = get_dataset_graph_cub(model_type,cub_groups,hierarchy.cub_attributes,sim_matrix
                                                                     , train, val)
                        
        hyperparameters = update_hyperparameters_graph(model_type,hyperparameters)
        
        model  = initialize_model(model_type,hyperparameters)
        model = train_model(model,model_type,train,val,hyperparameters)
        score = eval_model(model,model_type,val)
        score_list.append(score)
    
    scores_by_model_type[model_type] = np.mean(score_list)
    
    print("Score for {}, {} +- {}".format(model_type,np.mean(score_list),np.std(score_list)))

NameError: name 'cub_train_cem' is not defined

In [ ]:
scores_by_model_type

## 3-SAT Dataset

In [67]:
def run_model(model_type, hyperparameters, train, val, use_wandb=False):
    model = initialize_model(model_type,hyperparameters,use_wandb=use_wandb)
    model = train_model(model,model_type,train,val,hyperparameters,use_wandb=use_wandb)
    
    if use_wandb:
        wandb.finish()
    return model

In [74]:
num_variables = bottleneck_size = 30
sat_train, sat_val, clauses = create_3_sat_dataset(num_variables,5000,100)
output_classes = 2

In [75]:
hyperparameters = {'lr': .01, 'epochs': 200, 'num_layers': 1, 'emb_dim': 64, 
                       'in_dim': len(clauses)*len(clauses[0]), 'out_dim': output_classes, 
                       'edge_dim': 1,'clauses': clauses}
model_type = "mlp_group"

if 'gnn' in model_type:
    sat_edge_attr, sat_edge_index, sat_train_graph, sat_val_graph = get_dataset_graph(
        model_type, clauses, sat_train, sat_val)
    train = sat_train_graph
    val = sat_val_graph
else:
    train = sat_train
    val = sat_val

if 'mlp' in model_type:
    hyperparameters['in_dim'] = num_variables

hyperparameters = update_hyperparameters_graph(model_type,hyperparameters)

In [76]:
hyperparameters['epochs'] = 10

In [77]:
lr_values = [1e-4,5e-4,1e-3,5e-3,1e-2,5e-2,1e-1]

In [78]:
for lr in lr_values:
    hyperparameters['lr'] = lr
    model = run_model(model_type, hyperparameters, train, val, use_wandb=False)
    print("Eval {} {}".format(lr, eval_model(model,model_type,val)))

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type       | Params
---------------------------------------
0 | relu    | ReLU       | 0     
1 | softmax | Softmax    | 0     
2 | sigmoid | Sigmoid    | 0     
3 | fc      | Sequential | 1.3 K 
---------------------------------------
1.3 K     Trainable params
0         Non-trainable params
1.3 K     Total params
0.005     Total estimated model params size (MB)
`Trainer.fit` stopped: `max_epochs=10` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type       | Params
---------------------------------------
0 | relu    | ReLU       | 0     
1 | softmax | Softmax    | 0     
2 | sigmoid | Sigmoid    | 0     
3 | fc      | Sequential | 1.3 K 
---------------------------------------
1.3 K     Trainable params

Eval 0.0001 0.7300000190734863


`Trainer.fit` stopped: `max_epochs=10` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type       | Params
---------------------------------------
0 | relu    | ReLU       | 0     
1 | softmax | Softmax    | 0     
2 | sigmoid | Sigmoid    | 0     
3 | fc      | Sequential | 1.3 K 
---------------------------------------
1.3 K     Trainable params
0         Non-trainable params
1.3 K     Total params
0.005     Total estimated model params size (MB)


Eval 0.0005 0.7300000190734863


`Trainer.fit` stopped: `max_epochs=10` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type       | Params
---------------------------------------
0 | relu    | ReLU       | 0     
1 | softmax | Softmax    | 0     
2 | sigmoid | Sigmoid    | 0     
3 | fc      | Sequential | 1.3 K 
---------------------------------------
1.3 K     Trainable params
0         Non-trainable params
1.3 K     Total params
0.005     Total estimated model params size (MB)


...

Eval 0.01 0.6600000262260437


`Trainer.fit` stopped: `max_epochs=10` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type       | Params
---------------------------------------
0 | relu    | ReLU       | 0     
1 | softmax | Softmax    | 0     
2 | sigmoid | Sigmoid    | 0     
3 | fc      | Sequential | 1.3 K 
---------------------------------------
1.3 K     Trainable params
0         Non-trainable params
1.3 K     Total params
0.005     Total estimated model params size (MB)


Eval 0.05 0.7200000286102295


`Trainer.fit` stopped: `max_epochs=10` reached.


Eval 0.1 0.6800000071525574


In [73]:
hyperparameters['lr'] = 0.01
hyperparameters['epochs'] = 50
model = run_model(model_type, hyperparameters, train, val, use_wandb=True)

/rds/user/njr61/hpc-work/anaconda3/envs/gnn/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:396: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
/rds/user/njr61/hpc-work/anaconda3/envs/gnn/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:441: LightningDeprecationWarning: Setting `Trainer(gpus=0)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=0)` instead.
  rank_zero_deprecation(
/rds/user/njr61/hpc-work/anaconda3/envs/gnn/lib/python3.9/site-packages/lightning_lite/plugins/environments/slurm.py:167: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend y

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,▅▄▂▂▂▂▂▆█▂▃▄▃▂▂▃▁▂▃▂▂▂▂▁▂▁▁▂▂▂▂▂▂▂▂▁▁▁▁▁
train_acc,▁▃▄▃▄▄▄▃▂▂▂▃▄▄▅▆▆▇▇▇████████████████████
train_loss,▇▅▅▄▄▄▅▅██▇▆▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_acc,████████▆▄▃▂▂▃▃▂▃▃▃▄▄▃▃▃▃▄▄▄▄▅▄▄▃▁▂▁▂▂▂▂
val_loss,▁▁▂▃▅▆▇█▃▂▃▃▃▃▃▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇
epoch,49
loss,0.18527
train_acc,0.922
train_loss,0.22591


## CUB Dataset

In [118]:
bottleneck_size = 112
output_classes = 200

In [161]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
def merge_lists_of_lists(lists_of_lists, k):
    while len(lists_of_lists) > k:
        random.shuffle(lists_of_lists)  # shuffle the list of lists
        new_lists_of_lists = []
        for i in range(0, len(lists_of_lists) - 1, 2):
            new_list = lists_of_lists[i] + lists_of_lists[i+1]
            new_lists_of_lists.append(new_list)
        if len(lists_of_lists) % 2 == 1:
            new_lists_of_lists.append(lists_of_lists[-1])
        lists_of_lists = new_lists_of_lists
    return lists_of_lists


new_group_idx = merge_lists_of_lists(cub_groups_idx,2)

In [172]:
hyperparameters = {'lr': .01, 'epochs': 200, 'num_layers': 1, 'emb_dim': 64, 
                       'in_dim': bottleneck_size, 'out_dim': output_classes, 
                       'edge_dim': 1, 'sim_matrix': sim_matrix, 'attributes': hierarchy.cub_attributes, 'group': cub_groups, 
                       'indexes': cub_groups_idx}

model_type = "gnn_basic"
hyperparameters = update_hyperparameters_graph(model_type,hyperparameters)

In [177]:
train = cub_train_sequential
val = cub_val_sequential

if 'gnn' in model_type:
    edge_attr, edge_index, train, val = get_dataset_graph_cub(model_type,cub_groups,hierarchy.cub_attributes,sim_matrix
                                                             , train, val)

In [184]:
hyperparameters['epochs'] = 10

In [185]:
lr_values = [1e-2,5e-2,1e-1,5e-1]

In [186]:
for lr in lr_values:
    hyperparameters['lr'] = lr
    model = run_model(model_type, hyperparameters, train, val, use_wandb=False)
    print("Eval {} {}".format(lr, eval_model(model,model_type,val)))

Eval 0.01 0.27749061584472656
Eval 0.05 0.8632518649101257
Eval 0.1 0.8200187683105469
Eval 0.5 0.8127349615097046


In [187]:
hyperparameters['lr'] = 0.05
hyperparameters['epochs'] = 10

In [188]:
run_model(model_type, hyperparameters, train, val, use_wandb=True)

MPNNModel(
  (lin_in): Linear(in_features=112, out_features=64, bias=True)
  (convs): ModuleList(
    (0): MPNNLayer(emb_dim=64, aggr=add)
  )
  (group_pred): Linear(in_features=64, out_features=64, bias=True)
  (whole_pred): Linear(in_features=1984, out_features=64, bias=True)
  (lin_pred): Linear(in_features=64, out_features=200, bias=True)
)